In [1]:
import alpaca_trade_api as alpaca
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Replace YOUR_API_KEY and YOUR_API_SECRET with your Alpaca API key and secret
alpaca_api = alpaca.REST('PKU5LEIDZZV83Y5ENASI', 'nvzOwJpyvr73GTitTDyga0MkI2Qd6RsK4PmhItbu', api_version='v2')

# Set the ticker symbol and time frame
ticker = "AAPL"
timeframe = "1Day"

# Set the start and end dates for the data
start_date = "2012-01-01T00:00:00-00:00"
end_date = "2022-01-01T00:00:00-00:00"

# Retrieve the data
data = alpaca_api.get_bars(ticker,  timeframe, start_date, end_date).df

# Convert the data to a Pandas dataframe
df = pd.DataFrame(data)

# Calculate the 3 period moving average of the close
df['Mid']=(df['high']+df['low'])/2
#df['Mid']=df['high']
df['3MA'] = df['close'].rolling(3).mean()

df['2MA'] = df['close'].rolling(2).mean()

# Calculate the 10 period moving average of the close
df['10MA'] = df['close'].rolling(10).mean()

# Calculate the 20 period moving average of the close
#df['20MA'] = df['close'].rolling(20).mean()

df['Diff']=df['3MA']-df['10MA']

df['3MASlope']=df['3MA'].pct_change()
df['2MASlope']=df['2MA'].pct_change()
df['10MASlope']=df['10MA'].pct_change()
df['vwap']=df['vwap'].pct_change()
df['CloseDiff3MA']=df['close']/df['3MA']
df['open']=df['open'].pct_change().shift(-1)
df['close']=df['close'].pct_change().shift(-1)
df['Mid']=df['Mid'].pct_change()
df=df.drop(columns=['high','low','volume','trade_count'])
df['FutureMid']=df['Mid'].shift(-1)
df["Signal"] = 0.0
# Calculate the MACD(Moving Average Convergence Divergence) and Signal Line indicators
# 12-period EMA - 26-period EMA
df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
# Signal line 9-period EMA of the MACD
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()           



In [2]:
for index, row in df.iterrows():
    if row["Mid"] >= 0:
       df.loc[index, "Signal"] = 1.0
    if row["Mid"] < 0:
        df.loc[index,"Signal"] = -1.0

'''for index, row in df.iterrows():
    if row["close"] >= 0:
       df.loc[index, "Signal"] = 1.0
    if row["close"] < 0:
        df.loc[index,"Signal"] = 0'''
        
'''for index, row in df.iterrows():
    if row["vwap"] >= 0:
       df.loc[index, "Signal"] = 1.0
    if row["vwap"] < 0:
        df.loc[index,"Signal"] = 0.0'''
        
# Calculate the points in time at which a position should be taken, 1 or -1
#ema_signals_df['Crossover']= ema_signals_df['Signal'].diff()
#ema_signals_df=ema_signals_df.dropna()
#ema_signals_df['Crossover']=ema_signals_df['Crossover'].shift(-8)
df["Signal"]=df["Signal"].shift(-1)
df['FutureSlope']=df["close"].shift(-1)

'''ema_signals_df['Crossover']=ema_signals_df['Signal'].diff()
# Review the DataFrame
#ema_signals_df=ema_signals_df['Crossover'].dropna()
ema_signals_df=ema_signals_df.dropna()'''
df.tail(10)

,close,open,vwap,Mid,3MA,2MA,10MA,Diff,3MASlope,2MASlope,10MASlope,CloseDiff3MA,FutureMid,Signal,EMA_12,EMA_26,MACD,MACD_Signal,FutureSlope
timestamp,,,,,,,,,,,,,,,,,,,
2021-12-17 05:00:00+00:00,-0.008122,-0.009710,-0.017954,-0.024809,174.233333,171.700,173.836,0.397333,-0.006066,-0.023211,0.005379,0.982246,-0.014920,-1.0,-0.001256,0.002064,-0.003320,-0.000679,0.019087
2021-12-20 05:00:00+00:00,0.019087,0.019432,-0.011124,-0.014920,171.050000,170.445,174.279,-3.229000,-0.018271,-0.007309,0.002548,0.992400,0.012661,1.0,0.001874,0.003325,-0.001451,-0.000834,0.015319
2021-12-21 05:00:00+00:00,0.015319,0.008686,0.013512,0.012661,171.293333,171.370,174.460,-3.166667,0.001423,0.005427,0.001039,1.009905,0.016622,1.0,0.003942,0.004214,-0.000271,-0.000721,0.003644
2021-12-22 05:00:00+00:00,0.003644,0.016239,0.016429,0.016622,172.793333,174.315,174.516,-1.722667,0.008757,0.017185,0.000321,1.016474,0.011810,1.0,0.003896,0.004171,-0.000275,-0.000632,0.022975
2021-12-23 05:00:00+00:00,0.022975,0.007023,0.010118,0.011810,174.970000,175.960,174.688,0.282000,0.012597,0.009437,0.000986,1.007487,0.015251,1.0,0.006831,0.005564,0.001267,-0.000252,-0.005767
2021-12-27 05:00:00+00:00,-0.005767,0.017365,0.016845,0.015251,177.416667,178.305,174.776,2.640667,0.013983,0.013327,0.000504,1.016421,0.006630,1.0,0.004893,0.004725,0.000168,-0.000168,0.000502
2021-12-28 05:00:00+00:00,0.000502,-0.004607,0.003194,0.006630,178.633333,179.810,175.131,3.502333,0.006858,0.008441,0.002031,1.003676,-0.003029,-1.0,0.004218,0.004412,-0.000194,-0.000173,-0.006578
2021-12-29 05:00:00+00:00,-0.006578,0.000781,-0.001347,-0.003029,179.666667,179.335,175.636,4.030667,0.005785,-0.002642,0.002884,0.998404,-0.000307,-1.0,0.002557,0.003598,-0.001041,-0.000347,-0.003535
2021-12-30 05:00:00+00:00,-0.003535,-0.007717,-0.000664,-0.000307,178.956667,178.790,175.526,3.430667,-0.003952,-0.003039,-0.000626,0.995772,-0.006050,-1.0,0.001619,0.003070,-0.001450,-0.000568,NaN


In [3]:
#display(df.hvplot.scatter(y='CloseDiff3MA',x='3MASlope'))

lstColumns=df.columns

display(lstColumns)

dfopt=pd.DataFrame()

for column in lstColumns:
    display(df.hvplot.scatter(y='FutureMid',x=column))
    dfopt=df[['FutureMid',column]]
    display(dfopt.corr())

Index(['close', 'open', 'vwap', 'Mid', '3MA', '2MA', '10MA', 'Diff',
       '3MASlope', '2MASlope', '10MASlope', 'CloseDiff3MA', 'FutureMid',
       'Signal', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'FutureSlope'],
      dtype='object')

:Scatter   [close]   (FutureMid)

,FutureMid,close
FutureMid,1.000000,0.896017
close,0.896017,1.000000


:Scatter   [open]   (FutureMid)

,FutureMid,open
FutureMid,1.000000,0.913426
open,0.913426,1.000000


...

,FutureMid,MACD
FutureMid,1.000000,0.619797
MACD,0.619797,1.000000


:Scatter   [MACD_Signal]   (FutureMid)

,FutureMid,MACD_Signal
FutureMid,1.000000,0.227857
MACD_Signal,0.227857,1.000000


:Scatter   [FutureSlope]   (FutureMid)

,FutureMid,FutureSlope
FutureMid,1.000000,-0.055123
FutureSlope,-0.055123,1.000000


In [5]:
df=df.dropna()

y = df['Signal']

# Review the value counts
y.value_counts()

Signal
 1.0    858
-1.0    641
Name: count, dtype: int64

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Preparing data for Random Forest
X = df[['MACD', 'MACD_Signal', '10MASlope', '2MASlope', 'CloseDiff3MA']]  # Assuming these features are already calculated
y = df['Signal']  # Your target variable

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and fit Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)


In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the GridSearchCV object
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)

# Fit the grid search to the data
rf_grid_search.fit(X_train, y_train)

# Best parameters and model
best_grid_model = rf_grid_search.best_estimator_
print("Best parameters found: ", rf_grid_search.best_params_)


Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\nihar\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [8]:
# Assuming daily returns are already calculated in df['returns']
# Risk-free rate assumption
risk_free_rate = 0.01
# Calculate the excess returns by subtracting the risk-free rate from the returns
excess_returns = df['returns'] - risk_free_rate / 252

# Calculate the Sharpe Ratio
sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
print("Sharpe Ratio: ", sharpe_ratio)


KeyError: 'returns'